In [8]:
# %pip install "leafmap[maplibre]"

import os
import urllib.request, json 
import pandas as pd

In [9]:
import leafmap.maplibregl as maplibregl
import leafmap

In [10]:
import ipyleaflet
from PIL import Image
import requests
import math

# Save a map view in a PIL image
def toImage(m):

    # Bounds and zoom of the current view
    (latmin,lonmin),(latmax,lonmax) = m.bounds
    zoom = m.zoom

    # URLs of all the Tilelayer on the map
    baseUrls = [x.url for x in m.layers if type(x) == ipyleaflet.leaflet.TileLayer]

    # Opacities
    opacities = [x.opacity for x in m.layers if type(x) == ipyleaflet.leaflet.TileLayer]

    # Convert lat/lon/zoom to xtile,ytile.
    # See https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames
    def latlon2tile(lat_deg, lon_deg, zoom):
        lat_rad = (lat_deg * math.pi) / 180.0
        n = math.pow(2,zoom)
        xtile = n * ((lon_deg + 180.0) / 360.0)
        ytile = n * (1 - (math.log(math.tan(lat_rad) + 1.0/math.cos(lat_rad)) / math.pi)) / 2
        return xtile, ytile

    xtile1f,ytile2f = latlon2tile(latmin,lonmin, zoom)
    xtile2f,ytile1f = latlon2tile(latmax,lonmax, zoom)

    xtile1 = int(xtile1f)
    xtile2 = int(xtile2f)
    ytile1 = int(ytile1f)
    ytile2 = int(ytile2f)

    # Amount of pixels to crop on each side
    dx1 = 256*(xtile1f-xtile1)
    dx2 = 256*(xtile2+1-xtile2f)
    dy1 = 256*(ytile1f-ytile1)
    dy2 = 256*(ytile2+1-ytile2f)

    dx1 = round(dx1*100)//100
    dx2 = round(dx2*100)//100
    dy1 = round(dy1*100)//100
    dy2 = round(dy2*100)//100

    # Number of tiles
    nx = xtile2 - xtile1 + 1
    ny = ytile2 - ytile1 + 1

    # Dimension of the overall image
    w = 256 * nx
    h = 256 * ny
    imageTotal = Image.new(mode="RGBA", size=(w,h))

    # Substitute x,y,z into a TileService URL
    def url(baseurl, x,y,zoom):
        return baseurl.replace('{x}',str(int(x))).replace('{y}',str(int(y))).replace('{z}',str(int(zoom)))

    # Cycle on all tiles and compose the overall image
    for x in range(nx):
        xt = xtile1 + x
        xpos = x*256
        for y in range(ny):
            yt = ytile1 + y
            ypos = y*256
            for baseurl,opacity in zip(baseUrls,opacities):
                try:
                    image = Image.open(requests.get(url(baseurl,xt,yt,zoom), stream=True).raw)
                    image = image.convert('RGBA')

                    if opacity < 1.0:
                        # Split image in 4 channels
                        (r,g,b,a) = image.split()

                        # Change the alpha channel
                        # See https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.eval
                        a = Image.eval(a, lambda px: opacity*px)

                        # Merge 4 channels
                        image = Image.merge('RGBA',(r,g,b,a))

                    # Transparent paste!!!
                    # See https://stackoverflow.com/questions/5324647/how-to-merge-a-transparent-png-image-with-another-image-using-pil
                    imageTotal.paste(image, (xpos,ypos), mask=image)
                except:
                    pass

    # Crop the image
    area_crop = (dx1, dy1, w-dx2, h-dy2)
    return imageTotal.crop(area_crop)

In [11]:
# Load income data
INCOME_URL = "https://www.irs.gov/pub/irs-soi/21incyallagi.csv"
PD_INCOME = pd.read_csv(INCOME_URL, encoding='ISO-8859-1')

# Group by both STATEFIPS and COUNTYFIPS, and aggregate the N1 column
GROUPED = (
    PD_INCOME.groupby(["STATEFIPS", "COUNTYFIPS"], as_index=False)
    .agg(total_taxpayers=('N1', 'sum'))  # Sum N1 for each combination of STATEFIPS and COUNTYFIPS
)

# Convert the grouped DataFrame to a dictionary with a tuple (STATEFIPS, COUNTYFIPS) as keys
GROUPED_DICT = dict(zip(zip(GROUPED['STATEFIPS'], GROUPED['COUNTYFIPS']), GROUPED['total_taxpayers']))

# Calculate relative_percent using the composite key
PD_INCOME['relative_percent'] = PD_INCOME.apply(
    lambda row: 100 * row['N1'] / GROUPED_DICT.get((row["STATEFIPS"], row['COUNTYFIPS']), 1),
    axis=1
)

# Ensure two digits for STATEFIPS and five digits for COUNTYFIPS
PD_INCOME['STATEFIPS'] = PD_INCOME['STATEFIPS'].astype(int).astype(str).str.zfill(2)
PD_INCOME['COUNTYFIPS'] = PD_INCOME['COUNTYFIPS'].astype(int).astype(str).str.zfill(3)

# Create an 'id' by combining STATEFIPS and COUNTYFIPS
PD_INCOME['id'] = PD_INCOME['STATEFIPS'].astype(str) + PD_INCOME['COUNTYFIPS'].astype(str)

In [12]:
COUNTIES_URL = "https://open.gishub.org/data/us/us_counties.geojson"
with urllib.request.urlopen(COUNTIES_URL) as response:
    COUNTIES_JSON = json.load(response)
    PD_COUNTIES = pd.json_normalize(COUNTIES_JSON['features'])

# Iterate over the features in the GeoJSON
for feature in COUNTIES_JSON['features']:
    properties = feature.get('properties', {})
    state = properties.get('STATE')
    county = properties.get('COUNTY')
    
    if state is not None and county is not None:
        id = str(state) + str(county)  # Ensure both are strings
        if id:
            matching_rows = PD_INCOME[PD_INCOME['id'] == id]
            for _, row in matching_rows.iterrows():  # Use iterrows() to iterate over DataFrame rows
                properties[str(row['agi_stub']) + "total"] = row["N1"]  # Assign values to properties
                properties[str(row['agi_stub']) + "relative"] = row["relative_percent"]

In [13]:
nomansville = {
    "type": "Feature",
    "geometry": {
        "type": "MultiPolygon",
        "coordinates": [
            [
                [
                    ["-71.25", "36"],
                    ["-71.25", "37.75"],
                    ["-70", "37.75"],
                    ["-70", "36"],
                    ["-71.25", "36"]
                ]
            ]
        ]
    },
    "properties": {
        "GEO_ID": "0",
        "STATE": "01",
        "COUNTY": "000",
        "NAME": "Standardville",
        "LSAD": "County",
        "CENSUSAREA": "560.1",
        "1relative": 0,
        "1total": 500000,
        "2relative": 0,
        "2total": 500000,
        "3relative": 0,
        "3total": 500000,
        "4relative": 0,
        "4total": 500000,
        "5relative": 0,
        "5total": 500000,
        "6relative": 0,
        "6total": 500000,
        "7relative": 0,
        "7total": 500000,
        "8relative": 0,
        "8total": 500000
    }
}

# Append nomansville to the features array in COUNTIES_JSON
COUNTIES_JSON['features'].append(nomansville)

In [29]:
tax_bracket = "8"
# Add GeoJSON source
source = {
    "type": "geojson",
    "data": COUNTIES_JSON,
}

# Create a map
m = maplibregl.Map(center=[-95, 40], zoom=3.2,pitch=50, style="dark-matter")


m.add_source('counties', source)

# Define the fill-extrusion layer
layer = {
    "id": "us-counties",
    "source": "counties",
    "type": "fill-extrusion",
    "paint": {
        "fill-extrusion-color": [
            "interpolate",
            ["linear"],
            ["get", f"{tax_bracket}relative"],
            0, "#ffffff",
            25, "#88ff88",
            50, "#00ff00",
        ],
        "fill-extrusion-opacity": 1,
        "fill-extrusion-height": ["*", ["get", f"{tax_bracket}total"], 1],
    },
}

# Add the layer to the map
m.add_layer(layer)

# # Define the names layer
# names_layer = {
#     "id": "us-counties-name",
#     "source": "counties", 
#     "type": "symbol",
#     "layout": {
#         "text-field": ["get", "NAME"],  # Assuming 'NAME' is the field for county names
#         'text-variable-anchor': ['top', 'bottom', 'left', 'right'],
#         'text-radial-offset': 0.5,
#         'text-justify': 'auto',
#     }
# }

# # Add the names layer to the map
# m.add_layer(names_layer)

legend_dict = {
    f"0 % in AGI Stub {tax_bracket}": "ffffff",
    f"12.5 % in AGI Stub {tax_bracket}": "#BBffBB",
    f"25 % in AGI Stub {tax_bracket}": "#88ff88",
    f"37.5 % in AGI Stub {tax_bracket}": "#44ff44",
    f"50 % in AGI Stub {tax_bracket}": "00ff00",
}

leafmap.create_legend(
    title="Relative Percentage of Population",
    legend_dict=legend_dict,
    draggable=False,
    output="ESA_legend.html",
)
style = {
    "position": "fixed",
    "z-index": "9999",
    "border": "2px solid grey",
    "background-color": "rgba(255, 255, 255, 0.8)",
    "border-radius": "10px",
    "padding": "5px",
    "font-size": "14px",
    "bottom": "20px",
    "right": "5px",
}
m.add_legend(
    title="ESA Land Cover Type", legend_dict=legend_dict, draggable=False, style=style
)

places = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {"description": "50,0000 Returns", "icon": "theatre"},
            "geometry": {"type": "Point", "coordinates": [-70, 35.5]},
        },
    ],
}
source = {"type": "geojson", "data": places}
m.add_source("places", source)


layer = {
    "id": "poi-labels",
    "type": "symbol",
    "source": "places",
    "layout": {
        "text-field": ["get", "description"],
        "text-variable-anchor": ["top", "bottom", "left", "right"],
        "text-radial-offset": 0.5,
        "text-justify": "auto",
        "icon-image": ["concat", ["get", "icon"], "_15"]
    },
    "paint": {
        "text-color": "#FFFFFF"  # Sets the text color to white
    }
}

m.add_layer(layer)

# Add layer control
m.add_text("$200,000 under $500,000’", fontsize=12, bg_color="rgba(255, 255, 255, 0.8)", position="top-left")

# Display the map
m

Map(height='600px', map_options={'bearing': 0, 'center': (-95, 40), 'pitch': 50, 'style': 'https://basemaps.ca…